# Distirbuted Training of Mask-RCNN in Amazon SageMaker using FSx

This notebook is a step-by-step tutorial on distributed tranining of [Mask R-CNN]( https://arxiv.org/abs/1703.06870) implemented in [TensorFlow](https://www.tensorflow.org/) framework. Mask R-CNN is also referred to as heavy weight object detection model and it is part of [MLPerf](https://www.mlperf.org/training-results-0-6/).

Concretely, we will describe the steps for training [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) and [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) in [Amazon SageMaker](https://aws.amazon.com/sagemaker/) using [Amazon FSx for Lustre](https://aws.amazon.com/fsx/lustre/) file-system as data source.

The outline of steps is as follows:

1. Stage COCO 2017 dataset in [Amazon S3](https://aws.amazon.com/s3/)
2. Create Amazon FSx Lustre file-system and import data into the file-system from S3
3. Build Docker training image and push it to [Amazon ECR](https://aws.amazon.com/ecr/)
4. Configure data input channels
5. Configure hyper-prarameters
6. Define training metrics
7. Define training job and start training

Before we get started, let us initialize two python variables ```aws_region``` and ```s3_bucket``` that we will use throughout the notebook:

In [ ]:
aws_region =  # <aws-region>
s3_bucket =  # <your-s3_bucket>

## Stage COCO 2017 dataset in Amazon S3

We use [COCO 2017 dataset](http://cocodataset.org/#home) for training. We download COCO 2017 training and validation dataset to this notebook instance, extract the files from the dataset archives, and upload the extracted files to your Amazon [S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html). The ```prepare-s3-bucket.sh``` script executes this step. 

In [ ]:
!cat ./prepare-s3-bucket.sh

Using your *Amazon S3 bucket* as argument, run the cell below. If you have already uploaded COCO 2017 dataset to your Amazon S3 bucket, you may skip this step.

In [ ]:
%%time
!./prepare-s3-bucket.sh {s3_bucket}

## Create FSx Lustre file-system and import data from S3

Below, we use [AWS CloudFomration stack](https://docs.aws.amazon.com/en_pv/AWSCloudFormation/latest/UserGuide/stacks.html) to create a FSx Lustre file-system and import COCO 2017 dataset into the FSx file-system from your S3 bucket. 

In [ ]:
!cat stack-fsx.sh

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to Amazon CloudFormation and FSx services. If you created this notebook instance using the ```./stack-sm.sh``` script in this repository, the IAM Role attached to this notebook instance is already setup with requried access. 

```usage: ./stack-fsx.sh <aws-region> <s3-import-path> <fsx-capacity> <subnet-id> <security-group-id>```
    
<table>
    <caption>FSx configuraiton</caption>
    <tr>
        <th style="text-align:center">Argument</th>
        <th style="text-align:center">Description</th>
        <th style="text-align:center">Value</th>
    </tr>
    <tr>
        <td style="text-align:center">aws-region</td>
        <td style="text-align:left">AWS region name</td>
        <td style="text-align:center">e.g. us-east-1 </td>
     <tr>
    <tr>
        <td style="text-align:center">s3-import-path</td>
        <td style="text-align:left">S3 import path for importing data to FSx file-system</td>
        <td style="text-align:center">s3://&lt;s3-bucket-name&gt;/mask-rcnn/sagemaker/input</td>
     <tr>
      <tr>
        <td style="text-align:center">fsx-capacity</td>
        <td style="text-align:left">FSx Lustre file-system capacity in GiB</td>
        <td style="text-align:center">3600 or 7200</td>
     <tr>
      <tr>
        <td style="text-align:center">subnet-id</td>
        <td style="text-align:left">This is available in the output of ./stack-sm.sh script you used to create this notebook instance.  Specify only one subnet.</td>
        <td style="text-align:center">subnet-xxxx</td>
     <tr>
     <tr>
        <td style="text-align:center">security-group-id</td>
        <td style="text-align:left">Security group id for FSx lustre file system. This is available in the output of ./stack-sm.sh script you used to create this notebook instance.   </td>
        <td style="text-align:center">sg-xxxx</td>
     <tr>
</table>


If you have already created a FSx Lustre file-system and populated it with COCO 2017 dataset, you may skip this step.

In [ ]:
%%time
#usage: ./stack-fsx.sh <aws-region> <s3-import-path> <fsx-capacity> <subnet-id> <security-group-id>
!./stack-fsx.sh {aws_region} s3://{s3_bucket}/mask-rcnn/sagemaker/input 3600 'subet-xxxx' 'sg-xxxx'

## Build and push SageMaker training images

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to Amazon ECR service. If you created this notebook instance using the ```./stack-sm.sh``` script in this repository, the IAM Role attached to this notebook instance is already setup with full access to ECR service. 

Below, we have a choice of two different implementations:

1. [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) implementation supports a maximum per-GPU batch size of 1, and does not support mixed precision. It can be used with mainstream TensorFlow releases.

2. [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) is an optimized implementation that supports a maximum batch size of 4 and supports mixed precision. This implementation uses TensorFlow base version 1.13 augmented with custom TensorFlow ops. 

It is recommended that you build and push both SageMaker training images and use either image for training later.


### TensorPack Faster-RCNN/Mask-RCNN

Use ```./container/build_tools/build_and_push.sh``` script to build and push the TensorPack Faster-RCNN/Mask-RCNN  training image to Amazon ECR. 

In [ ]:
!cat ./container/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./container/build_tools/build_and_push.sh {aws_region}

Set ```tensorpack_image``` below to Amazon ECR URI of the image you pushed above.

In [ ]:
tensorpack_image = #<amazon-ecr-uri>

### AWS Samples Mask R-CNN
Use ```./container-optimized/build_tools/build_and_push.sh``` script to build and push the AWS Samples Mask R-CNN training image to Amazon ECR.

In [ ]:
!cat ./container-optimized/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./container-optimized/build_tools/build_and_push.sh {aws_region}

Set ```aws_samples_image``` below to the Amazon ECR URI of the image you pushed above.

In [ ]:
aws_samples_image = #<amazon-ecr-uri> 

## SageMaker Initialization 
We have staged the data and we have built and pushed the training docker image to Amazon ECR. Now we are ready to start using Amazon SageMaker.

In [ ]:
%%time
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

Next, we set ```training_image``` to the Amazon ECR image URI you saved in a previous step. 

In [ ]:
training_image = # set to tensorpack_image or aws_samples_image 
print(f'Training image: {training_image}')

## Define SageMaker Data Channels

Next, we define the *train* data channel using EFS file-system as data source. Set the FSx Lustre ```file_system_id``` prior to running the cell below.

In [ ]:
from sagemaker.inputs import FileSystemInput

# Specify FSx Lustre file system id.
file_system_id =  

# Specify directory path for input data on the file system. 
# You need to provide normalized and absolute path below.
file_system_directory_path = '/fsx/mask-rcnn/sagemaker/input/train'

# Specify the access mode of the mount of the directory associated with the file system. 
# Directory must be mounted 'ro'(read-only).
file_system_access_mode = 'ro'

# Specify your file system type.
file_system_type = 'FSxLustre'

train = FileSystemInput(file_system_id=file_system_id,
                                    file_system_type=file_system_type,
                                    directory_path=file_system_directory_path,
                                    file_system_access_mode=file_system_access_mode)

# Specify directory path for log output on the file system.
# This directory must exist, be empty and be writable
# You need to provide normalized and absolute path below.
# For example, '/fsx/mask-rcnn/sagemaker/output/log', 
# assuming this directory exists, is empty and is writeable
file_system_directory_path = 


# Specify the access mode of the mount of the directory associated with the file system. 
# Directory must be mounted 'rw'(read-write).
file_system_access_mode = 'rw'

log = FileSystemInput(file_system_id=file_system_id,
                                    file_system_type=file_system_type,
                                    directory_path=file_system_directory_path,
                                    file_system_access_mode=file_system_access_mode)

data_channels = {'train': train, 'log': log}

Next we define the model output location in S3. Set ```bucket``` to your S3 bucket name prior to running the cell below.

In [ ]:
prefix = "mask-rcnn/sagemaker" #prefix in your bucket
s3_output_location = f's3://{s3_bucket}/{prefix}/output'

## Configure Hyper-parameters
Next we define the hyper-parameters. 

Note, some hyper-parameters are different between the two implementations. The batch size per GPU in TensorPack Faster-RCNN/Mask-RCNN is fixed at 1, but is configurable in AWS Samples Mask-RCNN. The learning rate schedule is specified in units of steps in TensorPack Faster-RCNN/Mask-RCNN, but in epochs in AWS Samples Mask-RCNN.

The detault learning rate schedule values shown below correspond to training for a total of 24 epochs, at 120,000 images per epoch.

<table align='left'>
    <caption>TensorPack Faster-RCNN/Mask-RCNN  Hyper-parameters</caption>
    <tr>
    <th style="text-align:center">Hyper-parameter</th>
    <th style="text-align:center">Description</th>
    <th style="text-align:center">Default</th>
    </tr>
    <tr>
        <td style="text-align:center">mode_fpn</td>
        <td style="text-align:left">Flag to indicate use of Feature Pyramid Network (FPN) in the Mask R-CNN model backbone</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">mode_mask</td>
        <td style="text-align:left">A value of "False" means Faster-RCNN model, "True" means Mask R-CNN moodel</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">eval_period</td>
        <td style="text-align:left">Number of epochs period for evaluation during training</td>
        <td style="text-align:center">1</td>
    </tr>
    <tr>
        <td style="text-align:center">lr_schedule</td>
        <td style="text-align:left">Learning rate schedule in training steps</td>
        <td style="text-align:center">'[240000, 320000, 360000]'</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_norm</td>
        <td style="text-align:left">Batch normalization option ('FreezeBN', 'SyncBN', 'GN', 'None') </td>
        <td style="text-align:center">'FreezeBN'</td>
    </tr>
    <tr>
        <td style="text-align:center">images_per_epoch</td>
        <td style="text-align:left">Images per epoch </td>
        <td style="text-align:center">120000</td>
    </tr>
    <tr>
        <td style="text-align:center">data_train</td>
        <td style="text-align:left">Training data under data directory</td>
        <td style="text-align:center">'coco_train2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">data_val</td>
        <td style="text-align:left">Validation data under data directory</td>
        <td style="text-align:center">'coco_val2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">resnet_arch</td>
        <td style="text-align:left">Must be 'resnet50' or 'resnet101'</td>
        <td style="text-align:center">'resnet50'</td>
    </tr>
    <tr>
        <td style="text-align:center">backbone_weights</td>
        <td style="text-align:left">ResNet backbone weights</td>
        <td style="text-align:center">'ImageNet-R50-AlignPadding.npz'</td>
    </tr>
    <tr>
        <td style="text-align:center">load_model</td>
        <td style="text-align:left">Pre-trained model to load</td>
        <td style="text-align:center"></td>
    </tr>
    <tr>
        <td style="text-align:center">config:</td>
        <td style="text-align:left">Any hyperparamter prefixed with <b>config:</b> is set as a model config parameter</td>
        <td style="text-align:center"></td>
    </tr>
</table>

    
<table align='left'>
    <caption>AWS Samples Mask-RCNN  Hyper-parameters</caption>
    <tr>
    <th style="text-align:center">Hyper-parameter</th>
    <th style="text-align:center">Description</th>
    <th style="text-align:center">Default</th>
    </tr>
    <tr>
        <td style="text-align:center">mode_fpn</td>
        <td style="text-align:left">Flag to indicate use of Feature Pyramid Network (FPN) in the Mask R-CNN model backbone</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">mode_mask</td>
        <td style="text-align:left">A value of "False" means Faster-RCNN model, "True" means Mask R-CNN moodel</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">eval_period</td>
        <td style="text-align:left">Number of epochs period for evaluation during training</td>
        <td style="text-align:center">1</td>
    </tr>
    <tr>
        <td style="text-align:center">lr_epoch_schedule</td>
        <td style="text-align:left">Learning rate schedule in epochs</td>
        <td style="text-align:center">'[(16, 0.1), (20, 0.01), (24, None)]'</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_size_per_gpu</td>
        <td style="text-align:left">Batch size per gpu ( Minimum 1, Maximum 4)</td>
        <td style="text-align:center">4</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_norm</td>
        <td style="text-align:left">Batch normalization option ('FreezeBN', 'SyncBN', 'GN', 'None') </td>
        <td style="text-align:center">'FreezeBN'</td>
    </tr>
    <tr>
        <td style="text-align:center">images_per_epoch</td>
        <td style="text-align:left">Images per epoch </td>
        <td style="text-align:center">120000</td>
    </tr>
    <tr>
        <td style="text-align:center">data_train</td>
        <td style="text-align:left">Training data under data directory</td>
        <td style="text-align:center">'train2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">data_val</td>
        <td style="text-align:left">Validation data under data directory</td>
        <td style="text-align:center">'val2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">resnet_arch</td>
        <td style="text-align:left">Must be 'resnet50' or 'resnet101'</td>
        <td style="text-align:center">'resnet50'</td>
    </tr>
    <tr>
        <td style="text-align:center">backbone_weights</td>
        <td style="text-align:left">ResNet backbone weights</td>
        <td style="text-align:center">'ImageNet-R50-AlignPadding.npz'</td>
    </tr>
    <tr>
        <td style="text-align:center">load_model</td>
        <td style="text-align:left">Pre-trained model to load</td>
        <td style="text-align:center"></td>
    </tr>
    <tr>
        <td style="text-align:center">config:</td>
        <td style="text-align:left">Any hyperparamter prefixed with <b>config:</b> is set as a model config parameter</td>
        <td style="text-align:center"></td>
    </tr>
</table>

In [ ]:
hyperparameters = {
                    "mode_fpn": "True",
                    "mode_mask": "True",
                    "eval_period": 1,
                    "batch_norm": "FreezeBN"
                  }

## Define Training Metrics
Next, we define the regular expressions that SageMaker uses to extract algorithm metrics from training logs and send them to [AWS CloudWatch metrics](https://docs.aws.amazon.com/en_pv/AmazonCloudWatch/latest/monitoring/working_with_metrics.html). These algorithm metrics are visualized in SageMaker console.

In [ ]:
metric_definitions=[
             {
                "Name": "fastrcnn_losses/box_loss",
                "Regex": ".*fastrcnn_losses/box_loss:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_loss",
                "Regex": ".*fastrcnn_losses/label_loss:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/accuracy",
                "Regex": ".*fastrcnn_losses/label_metrics/accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/false_negative",
                "Regex": ".*fastrcnn_losses/label_metrics/false_negative:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/fg_accuracy",
                "Regex": ".*fastrcnn_losses/label_metrics/fg_accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/num_fg_label",
                "Regex": ".*fastrcnn_losses/num_fg_label:\\s*(\\S+).*"
            },
             {
                "Name": "maskrcnn_loss/accuracy",
                "Regex": ".*maskrcnn_loss/accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/fg_pixel_ratio",
                "Regex": ".*maskrcnn_loss/fg_pixel_ratio:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/maskrcnn_loss",
                "Regex": ".*maskrcnn_loss/maskrcnn_loss:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/pos_accuracy",
                "Regex": ".*maskrcnn_loss/pos_accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.5",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.5:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.5:0.95",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.5:0\\.95:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.75",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.75:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/large",
                "Regex": ".*mAP\\(bbox\\)/large:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/medium",
                "Regex": ".*mAP\\(bbox\\)/medium:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/small",
                "Regex": ".*mAP\\(bbox\\)/small:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.5",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.5:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.5:0.95",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.5:0\\.95:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.75",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.75:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/large",
                "Regex": ".*mAP\\(segm\\)/large:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/medium",
                "Regex": ".*mAP\\(segm\\)/medium:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/small",
                "Regex": ".*mAP\\(segm\\)/small:\\s*(\\S+).*"
            }  
            
    ]

## Define SageMaker Training Job

Next, we use SageMaker [Estimator](https://sagemaker.readthedocs.io/en/stable/estimators.html) API to define a SageMaker Training Job. 

We recommned using 32 GPUs, so we set ```train_instance_count=4``` and ```train_instance_type='ml.p3.16xlarge'```, because there are 8 Tesla V100 GPUs per ```ml.p3.16xlarge``` instance. We recommend using 100 GB [Amazon EBS](https://aws.amazon.com/ebs/) storage volume with each training instance, so we set ```train_volume_size = 100```. 

We run the training job in your private VPC, so we need to set the ```subnets``` and ```security_group_ids``` prior to running the cell below. You may specify multiple subnets in the ```subnets``` list. The subnets included in the ```sunbets``` list must be part of the output of  ```./stack-sm.sh``` CloudFormation stack script used to create this notebook instance. 

Specify one security group id in ```security_group_ids``` list. The security group id must be part of the output of  ```./stack-sm.sh``` CloudFormation stack script and be also attached to the FSx file-system created in a previous step.

For ```train_instance_type``` below, you have the option to use ```ml.p3.16xlarge``` with 16 GB per-GPU memory and 25 Gbs network interconnectivity, or ```ml.p3dn.24xlarge``` with 32 GB per-GPU memory and 100 Gbs network interconnectivity. The ```ml.p3dn.24xlarge``` instance type offers significantly better performance than ```ml.p3.16xlarge``` for Mask R-CNN distributed TensorFlow training.

In [ ]:

# Give Amazon SageMaker Training Jobs Access to FileSystem Resources in Your Amazon VPC.
security_group_ids =     # e.g ['sg-xxxxxx'] 
subnets =                # e.g. ['subnet-xxxxx']
sagemaker_session = sagemaker.session.Session(boto_session=session)

mask_rcnn_estimator = Estimator(training_image,
                                         role, 
                                         train_instance_count=4, 
                                         train_instance_type='ml.p3.16xlarge',
                                         train_volume_size = 100,
                                         train_max_run = 400000,
                                         output_path=s3_output_location,
                                         sagemaker_session=sagemaker_session, 
                                         hyperparameters = hyperparameters,
                                         metric_definitions = metric_definitions,
                                         base_job_name="mask-rcnn-fsxlustre",
                                         subnets=subnets,
                                         security_group_ids=security_group_ids)



Finally, we launch the SageMaker training job. 

The time to complete the training job depends on type and number of training instances, and the training image used for training.

In [ ]:
mask_rcnn_estimator.fit(inputs=data_channels, logs=True)